# Summary



# Intro

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from biodata import *

/home/kimlab1/strokach/anaconda3/lib/python3.5/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [3]:
from az_dream import functions as fn

In [4]:
from common import dat

In [5]:
%matplotlib inline

# Load data

In [6]:
input_data = {
    'scaled': {'raw': pd.read_hdf('machine_learning/ddc_data_scaled.h5')},
    'pca': {'raw': pd.read_hdf('machine_learning/ddc_data_pca.h5')},
    
}

## Make chunks

In [7]:
chunks = dict()
chunk_size = 900

In [8]:
def get_chunks(df_train):
    chunks = []
    df_train = df_train.reindex(np.random.permutation(df_train.index))
    for i in range(0, df_train.shape[0], chunk_size):
        train = pd.concat([df_train[0:i], df_train[i + chunk_size:]], ignore_index=True).copy()
        test = df_train[i:i + chunk_size].copy()
        #print(len(train))
        #print(len(test))

        # Make sure there is no same (d_1, d_2, c) in train and test
        test['unique_id'] = test[['d_1', 'd_2', 'c']].apply('.'.join, axis=1)
        train['unique_id'] = train[['d_1', 'd_2', 'c']].apply('.'.join, axis=1)
        test_unique_id = set(test['unique_id'])
        train = train[~train['unique_id'].isin(test_unique_id)]
        train_unique_id = set(train['unique_id'])
        #print(len(train))
        #print(len(test))
        assert not test_unique_id & train_unique_id, test_unique_id & train_unique_id

        # Make sure there are some (d_1, d_2) in train and test
        test_drug_pair = set(test[['d_1', 'd_2']].apply('.'.join, axis=1))
        train_drug_pair = set(train[['d_1', 'd_2']].apply('.'.join, axis=1))
        assert len(test_drug_pair & train_drug_pair) + 4 >= len(test_drug_pair), \
            (len(test_drug_pair & train_drug_pair), len(test_drug_pair))

        assert not train['synergy_score'].isnull().any()
        assert not test['synergy_score'].isnull().any()
        chunks.append((train, test))
    return chunks

In [10]:
for feature_format in ['scaled', 'pca']:
    input_df = input_data[feature_format]['raw']
    
    df_train = (
        input_df[
            (input_df['source'] == 'train') | 
            (input_df['source'] == 'ch2_validate')
        ]
    ).copy()

    df_validate = (
        input_df[
            (input_df['source'] == 'ch1_validate')
        ]
    ).copy()

    chunks = get_chunks(df_train)
    
    input_data[feature_format]['train'] = df_train
    input_data[feature_format]['train_chunks'] = chunks
    input_data[feature_format]['validate'] = df_validate
    # input_data[feature_format]['train_chunks'] = get_chunks(input_data[feature_format]['train'])

In [11]:
assert (df_train['d_1'].str.lower() <= df_train['d_2'].str.lower()).all()

# Tensorflow

In [13]:
N_COMPONENTS = 300

In [14]:
import tensorflow as tf

In [15]:
def model(data, train=False):
    relu = tf.nn.relu(tf.nn.bias_add(conv, conv1_biases))
    if train:
        hidden = tf.nn.dropout(hidden, 0.5, seed=SEED)
    tf.matmul(hidden, fc2_weights) + fc2_biases

In [16]:
def get_this_feature_columns(feature_format):    
    print("Input type: {}".format(feature_format))
    if feature_format == 'pca':
        this_feature_columns = list(range(N_COMPONENTS))
    else:
        this_feature_columns = feature_columns.copy()
    return this_feature_columns

In [22]:
# Linear regression

batch_size = 128
layer_1_size = 32
num_labels = 1

n_rows = None
n_columns = train_dataset.shape[1]
#n_columns = len(get_this_feature_columns(feature_format))

graph = tf.Graph()
with graph.as_default():
    tf.set_random_seed(42)
    
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(n_rows, n_columns))
    tf_train_labels = tf.placeholder(tf.float32, shape=(n_rows, num_labels))
    tf_valid_dataset = tf.placeholder(tf.float32, shape=(n_rows, n_columns))
    # tf_test_dataset = tf.placeholder(tf.float32, shape=(test_df.shape[0], len(feature_columns)))

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal([n_columns, layer_1_size], stddev=1))
    layer1_biases = tf.Variable(tf.zeros([layer_1_size]))
    layer2_weights = tf.Variable(tf.truncated_normal([layer_1_size, num_labels], stddev=1))
    layer2_biases = tf.Variable(tf.zeros([num_labels]))
    #layer3_weights = tf.Variable(tf.truncated_normal([256, num_labels]))
    #layer3_biases = tf.Variable(tf.zeros([num_labels]))
  
    def model(data, train=False):
        hidden = tf.matmul(data, layer1_weights) + layer1_biases
        hidden = tf.nn.relu6(hidden)
        if train:
            hidden = tf.nn.dropout(hidden, 0.5)
        hidden = tf.matmul(hidden, layer2_weights) + layer2_biases

        #hidden = tf.matmul(hidden, layer3_weights) + layer3_biases
        return hidden

    def correlation(X, Y):
        X_mean = tf.reduce_mean(X)
        Y_mean = tf.reduce_mean(Y)
        X_std = tf.sqrt(tf.reduce_sum(tf.square(X - X_mean)))
        Y_std = tf.sqrt(tf.reduce_sum(tf.square(Y - Y_mean)))
        cov = tf.matmul(tf.transpose(X - X_mean), (Y - Y_mean))
        corr = cov / (X_std * Y_std)
        return tf.squeeze(corr)
        
    
    logits = model(tf_train_dataset, train=True)
    #loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    #weights = (
    #    0.8 * tf.to_float(tf_train_labels > 20) + 
    #    0.2 * tf.to_float(tf_train_labels <= 20)
    #)
    #loss = tf.reduce_mean(tf.square(logits - tf_train_labels) * weights)
    #loss = tf.reduce_mean(tf.square(logits - tf_train_labels))
    loss = -correlation(logits, tf_train_labels)
    
    # Add regularization
    regularizers = (
        tf.nn.l2_loss(layer1_weights) + tf.nn.l2_loss(layer1_biases) +
        tf.nn.l2_loss(layer2_weights) + tf.nn.l2_loss(layer2_biases)
        #tf.nn.l2_loss(layer3_weights) + tf.nn.l2_loss(layer3_biases)
    )
    loss += 1e-5 * regularizers

    # Optimizer.
    step = tf.Variable(0, trainable=False)
    learning_rate = tf.train.exponential_decay(0.5, step, 1, 0.999)
    optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(loss)  # global_step=step)
    
    # Predictions for the training, validation, and test data.
    train_prediction = logits
    valid_prediction = model(tf_valid_dataset)
    # test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [23]:
from sklearn.utils import shuffle
from sklearn.preprocessing import Imputer, StandardScaler, RobustScaler, MaxAbsScaler
from sklearn.decomposition import PCA

imputer = Imputer(strategy='median')
standard_scaler = StandardScaler()
robust_scaler = RobustScaler()
maxabs_scaler = MaxAbsScaler()
pca = PCA()

scale_factor = 0.01

def get_data_for_step(feature_format, train_df, test_df):
    this_feature_columns = get_this_feature_columns(feature_format)
    
    if False:
        new_features = []
        
#         pca.fit(np.vstack([
#             train_sg[this_feature_columns + new_features].values, 
#             test_sg[this_feature_columns + new_features].values]))
#         train_dataset = (
#             pca.transform(train_df[this_feature_columns + new_features].values)[:, :N_COMPONENTS]
#         )
#         valid_dataset = (
#             pca.transform(test_df[this_feature_columns + new_features].values)[:, :N_COMPONENTS]
#         )
                
        train_dataset = train_df[this_feature_columns].values
        valid_dataset = test_df[this_feature_columns].values

    else:
        train_sg = fn.get_synergy_groups(train_df, train_df.drop('synergy_score', axis=1), pairwise=False)
        test_sg = fn.get_synergy_groups(train_df, test_df.drop('synergy_score', axis=1), pairwise=False)

        new_features = [
            c for c in train_sg.columns 
            if any([isinstance(c, str) and c.startswith(prefix) for prefix in ['count_', 'synergy_score_']])
        ]
        print(new_features)

        assert not any([
            c in (this_feature_columns + new_features)
            for c in ['synergy_score', 'synergy_score_diff']
        ])

        imputer.fit(np.vstack([train_sg[new_features].values, test_sg[new_features].values]))
        train_sg.loc[:, new_features] = imputer.transform(train_sg[new_features].values)
        test_sg.loc[:, new_features] = imputer.transform(test_sg[new_features].values)
        
        robust_scaler.fit(np.vstack([train_sg[new_features].values, test_sg[new_features].values]))
        train_sg.loc[:, new_features] = robust_scaler.transform(train_sg[new_features].values)
        test_sg.loc[:, new_features] = robust_scaler.transform(test_sg[new_features].values)

        maxabs_scaler.fit(np.vstack([train_sg[new_features].values, test_sg[new_features].values]))
        train_sg.loc[:, new_features] = maxabs_scaler.transform(train_sg[new_features].values)
        test_sg.loc[:, new_features] = maxabs_scaler.transform(test_sg[new_features].values)

        if True:
            pca.fit(np.vstack([
                train_sg[this_feature_columns + new_features].values, 
                test_sg[this_feature_columns + new_features].values]))
            train_dataset = (
                pca.transform(train_sg[this_feature_columns + new_features].values)[:, :N_COMPONENTS]
            )
            valid_dataset = (
                pca.transform(test_sg[this_feature_columns + new_features].values)[:, :N_COMPONENTS]
            )
        else:
            train_dataset = train_sg[this_feature_columns + new_features].values
            valid_dataset = test_sg[this_feature_columns + new_features].values
            
        assert (train_df[['d_1', 'd_2', 'c']] == train_sg[['d_1', 'd_2', 'c']]).all().all()
        assert (test_df[['d_1', 'd_2', 'c']] == test_sg[['d_1', 'd_2', 'c']]).all().all()
        
    train_labels = train_df[['synergy_score']].values * scale_factor
    valid_labels = test_df[['synergy_score']].values * scale_factor
    
    return train_dataset, train_labels, valid_dataset, valid_labels

In [30]:
num_steps = 103
output_dfs = []
feature_format = 'pca'

# for train_df, test_df in input_data[feature_format]['train_chunks']:
for train_df, test_df in [(input_data[feature_format]['train'], input_data[feature_format]['validate']), ]:
    
    train_df.index = range(len(train_df))
    test_df.index = range(len(test_df))
    grouped = shuffle(list(train_df.groupby(['d_1', 'd_2'])))
    grouped_c = shuffle(list(train_df.groupby(['c'])))
    
    train_dataset, train_labels, valid_dataset, valid_labels = (
        get_data_for_step(feature_format, train_df, test_df)
    )
    
    with tf.Session(graph=graph) as session:
        tf.initialize_all_variables().run()
        print("Initialized")
        
        for step in range(num_steps):
            batch_predictions_all = []
            batch_labels_all = []
            batch_losses = []

            which = np.random.choice([0, 1, 2])
            if (step % 50) in range(0, 3):
                which = step % 50
            
            which = 1
            
            if which == 0:
                train_dataset_this_step, train_labels_this_step = shuffle(train_dataset, train_labels)
                # Collect predictions from every batch
                for offset in range(0, train_dataset_this_step.shape[0] - batch_size , batch_size):
                    batch_data = train_dataset_this_step[offset:(offset + batch_size), :]
                    batch_labels = train_labels_this_step[offset:(offset + batch_size), :]
                    feed_dict = {
                        tf_train_dataset: batch_data, 
                        tf_train_labels: batch_labels,
                    }
                    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
                    #
                    batch_predictions_all.append(predictions)
                    batch_labels_all.append(batch_labels)
                    batch_losses.append(l)
            elif which == 1:              
                grouped = shuffle(grouped)
                n_skipped = 0
                for key, group in grouped:
                    if len(group) < 2:
                        n_skipped += 1
                        continue
                    batch_data = train_dataset[group.index, :]
                    batch_labels = train_labels[group.index, :]
                    feed_dict = {
                        tf_train_dataset: batch_data, 
                        tf_train_labels: batch_labels,
                    }
                    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
                    #
                    batch_predictions_all.append(predictions)
                    batch_labels_all.append(batch_labels)
                    batch_losses.append(l)
            elif which == 2:
                grouped_c = shuffle(grouped_c)
                n_skipped = 0
                for key, group in grouped_c:
                    if len(group) < 2:
                        n_skipped += 1
                        continue
                    batch_data = train_dataset[group.index, :]
                    batch_labels = train_labels[group.index, :]
                    feed_dict = {
                        tf_train_dataset: batch_data, 
                        tf_train_labels: batch_labels,
                    }
                    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
                    #
                    batch_predictions_all.append(predictions)
                    batch_labels_all.append(batch_labels)
                    batch_losses.append(l)

            step_predictions = np.vstack(batch_predictions_all)
            step_labels = np.vstack(batch_labels_all)
            step_loss = np.mean(batch_losses)
            
            if step % 20 == 0:
                print("Step: {} ({})".format(step, which))
                print("Minibatch loss at step {}: {}".format(step, l))
                if False:
                    print("Minibatch accuracy: %.2f%%" % accuracy(predictions, batch_labels))
                    print("Validation accuracy: %.2f%%" % accuracy(v_prediction, valid_labels))
                    fpr, tpr, thresholds = roc_curve(valid_labels[:, 0], v_prediction[:, 0])
                    bac = (tpr + (1 - fpr)) / 2
                    print('bac: {:.2f}\n'.format(max(bac)))
                else:
                    print("Minibatch correlation: {}".format(sp.stats.pearsonr(step_predictions, step_labels)[0]))
                    y_pred = valid_prediction.eval(feed_dict={tf_valid_dataset: valid_dataset})
                    print("Validation correlation: {}".format(sp.stats.pearsonr(y_pred, valid_labels)[0]))
                    tmp = test_df.copy()
                    tmp['synergy_score_pred'] = y_pred
                    print("Validation score: {}".format(fn.get_score_ch1(tmp)))
                print('')
                sys.stdout.flush()
                
        #saver.save(session, 'machine_learning/tensorflow_ch1', global_step=step)
        #print("Test accuracy: %.2f%%" % accuracy(test_prediction.eval(), test_labels))
    try:
        print("Number of groups: {}".format(len(grouped)))
        print("Skipped {} groups".format(n_skipped))
    except Exception as e:
        print(e)
    
    output_df = test_df[['d_1', 'd_2', 'c', 'synergy_score']].copy()
    output_df['synergy_score_pred'] = y_pred
    output_dfs.append(output_df)
    break

Input type: pca
['synergy_score_gbdd_median', 'count_gbdd', 'synergy_score_gbc_median', 'count_gbc', 'synergy_score_gbdc_median_mean', 'synergy_score_gbdc_median_diff', 'count_gbdc_mean', 'count_gbdc_diff', 'synergy_score_gbd_median_mean', 'synergy_score_gbd_median_diff', 'count_gbd_mean', 'count_gbd_diff']
Initialized
Step: 0 (1)
Minibatch loss at step 0: 0.1291968822479248
Minibatch correlation: [ 0.09575372]
Validation correlation: [ 0.16231606]
Skipped 0 out of 167 groups...
Validation score: (0.11943610009946184, 0.084153423464882485)

Step: 20 (1)
Minibatch loss at step 20: -0.8679263591766357
Minibatch correlation: [ 0.44471968]
Validation correlation: [ 0.2681028]
Skipped 0 out of 167 groups...
Validation score: (0.26024795117766553, 0.32079666138242707)

Step: 40 (1)
Minibatch loss at step 40: -0.7014860510826111
Minibatch correlation: [ 0.54281249]
Validation correlation: [ 0.25529122]
Skipped 0 out of 167 groups...
Validation score: (0.28367252535189696, 0.37101074889660091)

In [ ]:
plt.scatter(valid_labels, y_pred)

In [ ]:
tmp = df_validate.copy()
tmp['synergy_score_pred'] = y_pred
fn.get_score_ch1(tmp)